In [3]:
from selenium import webdriver
import pandas as pd
import os 

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# installation de driver 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


# chargement de la page de login
driver.get("https://www.linkedin.com/login")



# compte LinkedIn 
user_name = 'choukhantriikram@gmail.com'
password = 'chtr20082001'
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "username"))).send_keys(user_name)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "password"))).send_keys(password)

# boutton login         
elements = driver.find_elements('xpath', '//*[@id="organic-div"]/form/div[3]/button')
elements[0].click()

driver.implicitly_wait(10)

# page des jobs                          
WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="global-nav"]/div/nav/ul/li[3]/a'))).click()

# page d'ou on va scapper 
driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3324500841&geoId=102787409&keywords=business%20intelligence%20and%20analytics&location=Morocco&refresh=true")










# Get all links for these offers
links = []
# Navigate 13 pages
print('Links are being collected now.')

try: 
    for page in range(2,14):
        time.sleep(2)
        jobs_block = driver.find_element(By.CLASS_NAME, 'jobs-search-results-list')
        jobs_list = jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')
    
        for job in jobs_list:
            all_links = job.find_elements(By.TAG_NAME, 'a')
            for a in all_links:
                if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute('href') not in links: 
                    links.append(a.get_attribute('href'))
            # scroll down for each job element
            driver.execute_script("arguments[0].scrollIntoView();", job)
       
        print(f'Collecting the links on page {page-1}')
       # go to next page:
        driver.find_element(By.XPATH, f"//button[@aria-label='Page {page}']").click()
        time.sleep(3)

except Exception as e:
    print(f"An error occurred: {e}")                                     
   
    print('Found ' + str(len(links)) + ' links for job offers')

    

    
    

SyntaxError: invalid syntax (1640797683.py, line 56)

In [29]:
print(links)

['https://www.linkedin.com/jobs/view/3599036508/?eBP=JOB_SEARCH_ORGANIC&refId=IlUKUQdthOcaE9%2FhDd9ZVQ%3D%3D&trackingId=Thh4oKhfCtCsnKwLO4XHUg%3D%3D&trk=flagship3_search_srp_jobs', 'https://www.linkedin.com/jobs/view/3596291241/?eBP=JOB_SEARCH_ORGANIC&refId=IlUKUQdthOcaE9%2FhDd9ZVQ%3D%3D&trackingId=OAM%2Fa2FYs8WInBo6p2KI1Q%3D%3D&trk=flagship3_search_srp_jobs', 'https://www.linkedin.com/jobs/view/3608146573/?eBP=JOB_SEARCH_ORGANIC&refId=IlUKUQdthOcaE9%2FhDd9ZVQ%3D%3D&trackingId=Y7z2PnCOblB7CzNAR3xUvw%3D%3D&trk=flagship3_search_srp_jobs', 'https://www.linkedin.com/jobs/view/3552976177/?eBP=JOB_SEARCH_ORGANIC&refId=IlUKUQdthOcaE9%2FhDd9ZVQ%3D%3D&trackingId=H%2BiC7vWqGiUFCXb%2FeyOUZQ%3D%3D&trk=flagship3_search_srp_jobs', 'https://www.linkedin.com/jobs/view/3602570246/?eBP=JOB_SEARCH_ORGANIC&refId=IlUKUQdthOcaE9%2FhDd9ZVQ%3D%3D&trackingId=8iN1RU47FOIsnpiAwfzvbA%3D%3D&trk=flagship3_search_srp_jobs', 'https://www.linkedin.com/jobs/view/3610206071/?eBP=JOB_SEARCH_ORGANIC&refId=IlUKUQdthOcaE9%2

In [27]:
job_titles = []
company_names = []
company_locations = []
post_dates = []
job_desc = []
job_links = []



j = 1

# Visit each link one by one to scrape the information
print('Visiting the links and collecting information...')
for link in links:
    try:
        driver.get(link)
        time.sleep(2)
        # Click See more if available.
        see_more_button = driver.find_elements(By.XPATH, '//*[@id="ember33"]')
        see_more_button[0].click()
        time.sleep(2)
    except:
        pass

    # Find the general information of the job offers
    contents = driver.find_elements(By.CLASS_NAME, "p5")
    for content in contents:
        try:
            job_titles.append(content.find_element(By.CSS_SELECTOR, "h1").text)
            company_names.append(content.find_element(By.CSS_SELECTOR, ".jobs-unified-top-card__company-name").text)
            company_locations.append(content.find_element(By.CSS_SELECTOR, ".jobs-unified-top-card__bullet").text)
            post_dates.append(content.find_element(By.CSS_SELECTOR, ".jobs-unified-top-card__posted-date").text)
            print(f'Scraping job details: {j} DONE.')

            # Scraping the job description
            job_description = driver.find_elements(By.CLASS_NAME, "jobs-description__content")
            for description in job_description:
                job_text = description.text
                job_desc.append(job_text)
                job_links.append(link)
                print(f'Scraping the description of Job Offer {j}')
                time.sleep(2)

            j += 1
            time.sleep(2)

        except:
            pass

# Create separate DataFrames for each column of data
company_df = pd.DataFrame(company_names, columns=["Company"])
title_df = pd.DataFrame(job_titles, columns=["Job Title"])
date_df = pd.DataFrame(post_dates, columns=["Post Date"])
localisation_df = pd.DataFrame(company_locations, columns=["Location"])
description_df = pd.DataFrame(job_desc, columns=["Job Description"])
link_df = pd.DataFrame(job_links, columns=["Job Link"])

# Join the DataFrames into a single DataFrame
result_df = pd.concat([company_df, title_df, date_df, localisation_df,link_df, description_df], axis=1)
print(result_df)


Visiting the links and collecting information...
Scraping job details: 1 DONE.
Scraping the description of Job Offer 1
Scraping job details: 2 DONE.
Scraping the description of Job Offer 2
Scraping job details: 3 DONE.
Scraping the description of Job Offer 3
Scraping job details: 4 DONE.
Scraping the description of Job Offer 4
Scraping job details: 5 DONE.
Scraping the description of Job Offer 5
Scraping job details: 6 DONE.
Scraping the description of Job Offer 6
Scraping job details: 7 DONE.
Scraping the description of Job Offer 7
Scraping job details: 8 DONE.
Scraping the description of Job Offer 8
Scraping job details: 9 DONE.
Scraping the description of Job Offer 9
Scraping job details: 10 DONE.
Scraping the description of Job Offer 10
Scraping job details: 11 DONE.
Scraping the description of Job Offer 11
Scraping job details: 12 DONE.
Scraping the description of Job Offer 12
Scraping job details: 13 DONE.
Scraping the description of Job Offer 13
Scraping job details: 14 DONE.
Sc

Scraping job details: 114 DONE.
Scraping the description of Job Offer 114
Scraping job details: 115 DONE.
Scraping the description of Job Offer 115
Scraping job details: 116 DONE.
Scraping the description of Job Offer 116
Scraping job details: 117 DONE.
Scraping the description of Job Offer 117
Scraping job details: 118 DONE.
Scraping the description of Job Offer 118
Scraping job details: 119 DONE.
Scraping the description of Job Offer 119
Scraping job details: 120 DONE.
Scraping the description of Job Offer 120
Scraping job details: 121 DONE.
Scraping the description of Job Offer 121
Scraping job details: 122 DONE.
Scraping the description of Job Offer 122
Scraping job details: 123 DONE.
Scraping the description of Job Offer 123
Scraping job details: 124 DONE.
Scraping the description of Job Offer 124
Scraping job details: 125 DONE.
Scraping the description of Job Offer 125
Scraping job details: 126 DONE.
Scraping the description of Job Offer 126
Scraping job details: 127 DONE.
Scrapi

In [1]:
result_df.to_excel('result.xlsx', index=False)

NameError: name 'result_df' is not defined

In [12]:
import pandas as pd
import re

def extract_phrases_with_words(text, words):
    pattern = r"(.*?[.!?])"
    phrases = []
    matches = re.findall(pattern, text, re.IGNORECASE | re.MULTILINE)
    
    for match in matches:
        if any(re.search(r"\b" + re.escape(word) + r"\b", match, re.IGNORECASE) for word in words):
            phrases.append(match.strip())
    
    return phrases

def extract_words(text, words):
    words_found = []
    
    for word in words:
        if re.search(r"\b" + re.escape(word) + r"\b", text, re.IGNORECASE):
            words_found.append(word)
    
    return words_found

# Charger le fichier Excel
data_frame = pd.read_excel('C:\\Users\\pc\\Desktop\\finalresult.xlsx')

# Nom de la feuille de calcul à utiliser
nom_feuille = 'Sheet1'

# Colonnes à parcourir dans le fichier Excel
colonnes = ['Job Description']

# Parcourir chaque ligne du fichier Excel
for index, row in data_frame.iterrows():
    traitement_phrases1 = ''
    traitement_phrases2 = ''
    traitement_sql = ''
    traitement_MySql = ''
    traitement_PostgreSql = ''
    traitement_SqlServer = ''
    traitement_PowerBI = ''
    traitement_tableau = ''
    traitement_Qlik = ''
    traitement_BigData = ''
    traitement_Cloudera = ''
    traitement_hive = ''
    traitement_Pyspark = ''
    traitement_Impala = ''
    traitement_DataScience = ''
    traitement_Dataiku = ''
    traitement_Knime = ''
    traitement_IBMSPSS = ''
    traitement_ETL = ''
    traitement_Talend = ''
    traitement_Informatica = ''
    traitement_SSIS = ''
    traitement_DataAnalytics = ''
    traitement_python = ''
    traitement_Pandas = ''
    traitement_NumPy = ''
    traitement_R = ''
    traitement_java = ''
    traitement_C = ''
    traitement_C_plus = ''
    traitement_java_avance = ''
    
    for colonne in colonnes:
        cellule = str(row[colonne])
        phrases_francais_anglais = extract_phrases_with_words(cellule, ['français', 'anglais'])
        phrase_etu = extract_phrases_with_words(cellule, ['école', 'bac+'])
        mots_sql = extract_words(cellule, ['SQL'])
        mots_MySQL = extract_words(cellule, ['MySQL'])
        mots_PostgreSql = extract_words(cellule, ['PostgreSql'])
        mots_SqlServer = extract_words(cellule, ['SQL Server'])
        mots_PowerBI = extract_words(cellule, ['Power BI'])
        mots_tableau = extract_words(cellule, ['tableau'])
        mots_Qlik = extract_words(cellule, ['Qlik'])
        mots_BigData = extract_words(cellule, ['Big Data'])
        mots_Cloudera = extract_words(cellule, ['Cloudera'])
        mots_hive = extract_words(cellule, ['hive'])
        mots_Pyspark = extract_words(cellule, ['Pyspark'])
        mots_Impala = extract_words(cellule, ['Impala'])
        mots_DataScience = extract_words(cellule, ['Data Science'])
        mots_Dataiku = extract_words(cellule, ['Dataiku'])
        mots_Knime = extract_words(cellule, ['Knime'])
        mots_IBMSPSS = extract_words(cellule, ['IBM SPSS'])
        mots_ETL = extract_words(cellule, ['ETL'])
        mots_Talend = extract_words(cellule, ['Talend'])
        mots_Informatica = extract_words(cellule, ['Informatica'])
        mots_SSIS = extract_words(cellule, ['SSIS'])
        mots_DataAnalytics = extract_words(cellule, ['Data analytics'])
        mots_python = extract_words(cellule, ['python'])
        mots_Pandas = extract_words(cellule, ['Pandas'])
        mots_NumPy = extract_words(cellule, ['NumPy'])
        mots_R = extract_words(cellule, ['R'])
        mots_java = extract_words(cellule, ['java'])
        mots_C = extract_words(cellule, ['C'])
        mots_C_plus = extract_words(cellule, ['C++'])
        mots_java_avance = extract_words(cellule, ['java avancé'])
        
        if phrases_francais_anglais:
            traitement_phrases1 += ' '.join(phrases_francais_anglais) + ' '
        
        if phrase_etu:
            traitement_phrases2 += ' '.join(phrase_etu) + ' '
        
        if mots_sql:
            traitement_sql += ' '.join(mots_sql) + ' '
        
        if mots_MySQL:
            traitement_MySql += ' '.join(mots_MySQL) + ' '
        
        if mots_PostgreSql:
            traitement_PostgreSql += ' '.join(mots_PostgreSql) + ' '
        
        if mots_SqlServer:
            traitement_SqlServer += ' '.join(mots_SqlServer) + ' '
        
        if mots_PowerBI:
            traitement_PowerBI += ' '.join(mots_PowerBI) + ' '
        
        if mots_tableau:
            traitement_tableau += ' '.join(mots_tableau) + ' '
        
        if mots_Qlik:
            traitement_Qlik += ' '.join(mots_Qlik) + ' '
        
        if mots_BigData:
            traitement_BigData += ' '.join(mots_BigData) + ' '
        
        if mots_Cloudera:
            traitement_Cloudera += ' '.join(mots_Cloudera) + ' '
        
        if mots_hive:
            traitement_hive += ' '.join(mots_hive) + ' '
        
        if mots_Pyspark:
            traitement_Pyspark += ' '.join(mots_Pyspark) + ' '
        
        if mots_Impala:
            traitement_Impala += ' '.join(mots_Impala) + ' '
        
        if mots_DataScience:
            traitement_DataScience += ' '.join(mots_DataScience) + ' '
        
        if mots_Dataiku:
            traitement_Dataiku += ' '.join(mots_Dataiku) + ' '
        
        if mots_Knime:
            traitement_Knime += ' '.join(mots_Knime) + ' '
        
        if mots_IBMSPSS:
            traitement_IBMSPSS += ' '.join(mots_IBMSPSS) + ' '
        
        if mots_ETL:
            traitement_ETL += ' '.join(mots_ETL) + ' '
        
        if mots_Talend:
            traitement_Talend += ' '.join(mots_Talend) + ' '
        
        if mots_Informatica:
            traitement_Informatica += ' '.join(mots_Informatica) + ' '
        
        if mots_SSIS:
            traitement_SSIS += ' '.join(mots_SSIS) + ' '
        
        if mots_DataAnalytics:
            traitement_DataAnalytics += ' '.join(mots_DataAnalytics) + ' '
        
        if mots_python:
            traitement_python += ' '.join(mots_python) + ' '
        
        if mots_Pandas:
            traitement_Pandas += ' '.join(mots_Pandas) + ' '
        
        if mots_NumPy:
            traitement_NumPy += ' '.join(mots_NumPy) + ' '
        
        if mots_R:
            traitement_R += ' '.join(mots_R) + ' '
        
        if mots_java:
            traitement_java += ' '.join(mots_java) + ' '
        
        if mots_C:
            traitement_C += ' '.join(mots_C) + ' '
        
        if mots_C_plus:
            traitement_C_plus += ' '.join(mots_C_plus) + ' '
        
        if mots_java_avance:
            traitement_java_avance += ' '.join(mots_java_avance) + ' '

    # Enregistrer le traitement dans la colonne "Traitement"
    data_frame.at[index, 'langue'] = traitement_phrases1
    data_frame.at[index, 'diplome'] = traitement_phrases2
    data_frame.at[index, 'SQL'] = traitement_sql
    data_frame.at[index, 'MySQL'] = traitement_MySql
    data_frame.at[index, 'PostgreSql'] = traitement_PostgreSql
    data_frame.at[index, 'SQL Server'] = traitement_SqlServer
    data_frame.at[index, 'Power BI'] = traitement_PowerBI
    data_frame.at[index, 'tableau'] = traitement_tableau
    data_frame.at[index, 'Qlik'] = traitement_Qlik
    data_frame.at[index, 'Big Data'] = traitement_BigData
    data_frame.at[index, 'Cloudera'] = traitement_Cloudera
    data_frame.at[index, 'hive'] = traitement_hive
    data_frame.at[index, 'Pyspark'] = traitement_Pyspark
    data_frame.at[index, 'Impala'] = traitement_Impala
    data_frame.at[index, 'Data Science'] = traitement_DataScience
    data_frame.at[index, 'Dataiku'] = traitement_Dataiku
    data_frame.at[index, 'Knime'] = traitement_Knime
    data_frame.at[index, 'IBM SPSS'] = traitement_IBMSPSS
    data_frame.at[index, 'ETL'] = traitement_ETL
    data_frame.at[index, 'Talend'] = traitement_Talend
    data_frame.at[index, 'Informatica'] = traitement_Informatica
    data_frame.at[index, 'SSIS'] = traitement_SSIS
    data_frame.at[index, 'Data analytics'] = traitement_DataAnalytics
    data_frame.at[index, 'python'] = traitement_python
    data_frame.at[index, 'Pandas'] = traitement_Pandas
    data_frame.at[index, 'NumPy'] = traitement_NumPy
    data_frame.at[index, 'R'] = traitement_R
    data_frame.at[index, 'java'] = traitement_java
    data_frame.at[index, 'C'] = traitement_C
    data_frame.at[index, 'C++'] = traitement_C_plus
    data_frame.at[index, 'java avancé'] = traitement_java_avance

# Enregistrer les modifications dans le fichier Excel
data_frame.to_excel('C:\\Users\\pc\\Desktop\\finalresult.xlsx', index=False, sheet_name=nom_feuille)



           



In [45]:
import pandas as pd
from datetime import datetime, timedelta
import locale
import re

# Lecture du fichier Excel
df = pd.read_excel(r"C:\Users\pc\Desktop\PFA\sheets\jobs\result.xlsx")

# Nom de la colonne contenant les informations de durée
colonne_duree = "Post Date"

# Configuration des paramètres régionaux pour le format de date
locale.setlocale(locale.LC_ALL, '')

# Date ultérieure donnée
date_ulterieure = datetime(2023, 5, 30, 23, 23, 55)

# Fonction pour convertir la durée en date de publication par rapport à la date ultérieure
def convertir_duree_en_date(duree):
    if duree is None or pd.isnull(duree):
        return None

    if isinstance(duree, str):
        match = re.search(r"il y a (\d+) (\w+)", duree)
        if match:
            quantite = int(match.group(1))
            unite = match.group(2)

            if unite == "heure" or unite == "heures":
                date_publication = date_ulterieure - timedelta(hours=quantite)
            elif unite == "semaine" or unite == "semaines":
                date_publication = date_ulterieure - timedelta(weeks=quantite)
            elif unite == "mois":
                date_publication = date_ulterieure - timedelta(days=30 * quantite)
            else:
                return None

            return date_publication.strftime("%A %d %B %Y, %H:%M:%S")

    return None

# Conversion des durées en dates de publication
df["Date de publication"] = df[colonne_duree].apply(lambda x: convertir_duree_en_date(x) if x is not None and not pd.isnull(x) else None)

# Affichage du dataframe avec la nouvelle colonne de dates de publication
df.to_excel("result-date-trait.xlsx", index=False)


In [49]:
import pandas as pd

# Lecture du fichier Excel
df = pd.read_excel(r"C:\Users\pc\Desktop\PFA\sheets\jobs\result-date-trait.xlsx")

# Traitement de la colonne "Location"
df.loc[df["Location"].str.contains("casablanca", case=False), "Location"] = "Casablanca et périphérie"
df.loc[df["Location"].str.contains("rabat", case=False), "Location"] = "Rabat et périphérie"

# Enregistrement du dataframe dans un fichier Excel
df.to_excel("result-loc-trait.xlsx", index=False)


In [3]:
import pandas as pd

mots_cles = ["Python", "Pandas", "Numpy","Beautiful soup","Selenium", "SQL Server", "PostgreSql", "MySQL", "SQL", "NO SQL", "PL/SQL", "Linux", "SGBD",
             "Git", "Qlik", "tableau", "Power BI", "Power Query", "Latex", "Qlick view", "Cloudera", "TenserFlow","SAP","SSIS"
             "OpenCv", "Matlab", "Apache", "Jira", "C++", "JAVA", "java", "Merise", "HTML", "CSS", "Laravel","PHP","Pyspark","ETL","Dataiku",
             "Jquerry", "UML", "Data analytics", "Statistics", "Problem Solving", "Web Scraping", "Data Modeling","Knime","Impala","e-commerce"
             "Artificial Intelligence", "Data Engineering", "Data Analytics", "Data Warehouse", "Applied Mathematics","datamining",
             "Big Data", "Data Visualisation", "Business Intelligence", "Machine Learning", "ERP","ANAPLAN", "CRM", "GRH", "PMO" , "Microsoft Excel"]

def extract_keywords(text):
    keywords = []
    for mot_cle in mots_cles:
        if mot_cle.lower() in text.lower():
            keywords.append(mot_cle)
    return ", ".join(keywords)

# Lire le fichier Excel
df = pd.read_excel( r"C:\Users\pc\Desktop\PFA\sheets\jobs\result-loc-trait.xlsx")

# Appliquer la fonction extract_keywords pour créer la colonne "compétences"
df["compétences"] = df["Job Description"].apply(extract_keywords)

# Enregistrer le DataFrame modifié dans un nouveau fichier Excel
df.to_excel("CASE-VIDE.xlsx", index=False)
